# 🔧 PACE OCI - Notebook de Diagnóstico

Execute célula por célula para identificar onde está o problema.

In [15]:
# CÉLULA 1: Instalar dependências
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", 
                       "earthaccess", "xarray", "netCDF4", "h5netcdf", "--quiet"])
print("✓ Pacotes instalados")

✓ Pacotes instalados


In [16]:
# CÉLULA 2: Imports e verificação de versão
import earthaccess
import xarray as xr
from pathlib import Path

print(f"earthaccess version: {earthaccess.__version__}")
print(f"xarray version: {xr.__version__}")

earthaccess version: 0.15.1
xarray version: 2025.12.0


In [17]:
# CÉLULA 3: Verificar credenciais salvas
print("="*60)
print("VERIFICANDO CREDENCIAIS SALVAS")
print("="*60)

netrc_path = Path.home() / '.netrc'
print(f"\nPath ~/.netrc: {netrc_path}")
print(f"Existe: {netrc_path.exists()}")

if netrc_path.exists():
    with open(netrc_path) as f:
        content = f.read()
        if 'urs.earthdata.nasa.gov' in content:
            print("✓ Credenciais Earthdata encontradas em ~/.netrc")
            # Show username (but not password)
            for line in content.split('\n'):
                if 'urs.earthdata.nasa.gov' in line:
                    print(f"  {line.split('login')[0]}login ***")
        else:
            print("⚠ ~/.netrc existe mas NÃO contém credenciais Earthdata")
else:
    print("⚠ ~/.netrc NÃO existe - credenciais não salvas")

VERIFICANDO CREDENCIAIS SALVAS

Path ~/.netrc: /home/desenvolvedor/.netrc
Existe: True
✓ Credenciais Earthdata encontradas em ~/.netrc
  machine urs.earthdata.nasa.govlogin ***


In [18]:
# CÉLULA 4: Tentar autenticação automática
print("="*60)
print("TENTANDO AUTENTICAÇÃO")
print("="*60)

try:
    auth = earthaccess.login()
    print(f"\nResultado: {auth}")
    print(f"Tipo: {type(auth)}")
    
    # Check store
    if earthaccess.__store__ is not None:
        print("\n✓ AUTENTICAÇÃO OK!")
        print("  Store inicializado corretamente")
    else:
        print("\n⚠ Store é None - autenticação pode ter falhado")
        
except Exception as e:
    print(f"\n✗ Erro na autenticação: {e}")
    print("\n→ Execute a próxima célula para autenticação interativa")

TENTANDO AUTENTICAÇÃO

Resultado: <earthaccess.auth.Auth object at 0x797ff80ae6f0>
Tipo: <class 'earthaccess.auth.Auth'>

✓ AUTENTICAÇÃO OK!
  Store inicializado corretamente


In [ ]:
# CÉLULA 5: AUTENTICAÇÃO INTERATIVA
# ================================
# Descomente e execute se a célula anterior falhou
# Você precisará inserir seu usuário e senha NASA Earthdata

# auth = earthaccess.login(strategy='interactive')
# print(f"Auth: {auth}")
# print(f"Store: {earthaccess.__store__ is not None}")

In [19]:
# CÉLULA 6: Testar busca de dados
print("="*60)
print("TESTANDO BUSCA DE DADOS")
print("="*60)

test_date = "2024-06-15"
short_name = "PACE_OCI_L3M_CHL"

print(f"\nBuscando {short_name} para {test_date}...")

try:
    results = earthaccess.search_data(
        short_name=short_name,
        temporal=(test_date, test_date),
        granule_name="*.DAY.*.0p1deg.*"
    )
    
    print(f"✓ Encontrados: {len(results)} arquivo(s)")
    
    if results:
        for r in results:
            url = r.data_links()[0]
            print(f"  → {url.split('/')[-1]}")
            print(f"    URL: {url[:80]}...")
    else:
        print("⚠ Nenhum arquivo encontrado para esta data")
        
except Exception as e:
    print(f"✗ Erro na busca: {e}")
    results = []

TESTANDO BUSCA DE DADOS

Buscando PACE_OCI_L3M_CHL para 2024-06-15...
✓ Encontrados: 1 arquivo(s)
  → PACE_OCI.20240615.L3m.DAY.CHL.V3_1.chlor_a.0p1deg.nc
    URL: https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240...


In [20]:
# CÉLULA 7: Testar abertura de arquivo (PASSO CRÍTICO)
print("="*60)
print("TESTANDO ABERTURA DE ARQUIVO")
print("="*60)

if not results:
    print("⚠ Execute a célula anterior primeiro (busca)")
else:
    print(f"\nTentando abrir com earthaccess.open()...")
    
    try:
        fileset = earthaccess.open(results)
        print(f"✓ Fileset criado!")
        print(f"  Tipo: {type(fileset)}")
        print(f"  Quantidade: {len(fileset)}")
        print(f"  Item type: {type(fileset[0])}")
        
    except AttributeError as e:
        if "'NoneType'" in str(e):
            print("")
            print("="*60)
            print("✗ ERRO DE AUTENTICAÇÃO DETECTADO!")
            print("="*60)
            print("""
O earthaccess.__store__ é None, indicando que a 
autenticação não foi concluída corretamente.

SOLUÇÃO:
1. Volte à célula 5 e descomente as linhas
2. Execute para autenticar interativamente
3. Insira suas credenciais NASA Earthdata
4. Execute esta célula novamente

Se não tem conta, registre em:
https://urs.earthdata.nasa.gov
""")
        else:
            print(f"✗ Erro: {e}")
            
    except Exception as e:
        print(f"✗ Erro: {e}")
        import traceback
        traceback.print_exc()

TESTANDO ABERTURA DE ARQUIVO

Tentando abrir com earthaccess.open()...


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

✓ Fileset criado!
  Tipo: <class 'list'>
  Quantidade: 1
  Item type: <class 'earthaccess.store.EarthAccessFile'>


In [21]:
# CÉLULA 8: Testar leitura com xarray
print("="*60)
print("TESTANDO LEITURA COM XARRAY")
print("="*60)

try:
    print("\nAbrindo dataset com xarray...")
    ds = xr.open_dataset(fileset[0])
    
    print(f"\n✓ Dataset aberto com sucesso!")
    print(f"\nDimensões: {dict(ds.dims)}")
    print(f"Variáveis: {list(ds.data_vars)}")
    print(f"Lat range: {float(ds.lat.min()):.1f} a {float(ds.lat.max()):.1f}")
    print(f"Lon range: {float(ds.lon.min()):.1f} a {float(ds.lon.max()):.1f}")
    
    # Subset test
    print(f"\nTestando subset para Cape Hatteras...")
    ds_sub = ds.sel(
        lat=slice(50, 30),
        lon=slice(-80, -60)
    )
    print(f"Dimensões subset: {dict(ds_sub.dims)}")
    print(f"Tamanho estimado: {ds_sub.nbytes / 1e6:.1f} MB")
    
    ds.close()
    print("\n✓ TUDO FUNCIONANDO!")
    
except NameError:
    print("⚠ Execute a célula anterior primeiro (fileset não definido)")
except Exception as e:
    print(f"✗ Erro: {e}")
    import traceback
    traceback.print_exc()

TESTANDO LEITURA COM XARRAY

Abrindo dataset com xarray...

✓ Dataset aberto com sucesso!

Dimensões: {'lat': 1800, 'lon': 3600, 'rgb': 3, 'eightbitcolor': 256}
Variáveis: ['chlor_a', 'palette']
Lat range: -90.0 a 89.9
Lon range: -179.9 a 180.0

Testando subset para Cape Hatteras...
Dimensões subset: {'lat': 200, 'lon': 200, 'rgb': 3, 'eightbitcolor': 256}
Tamanho estimado: 0.2 MB

✓ TUDO FUNCIONANDO!


/tmp/ipykernel_3403848/4227912475.py:11: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"\nDimensões: {dict(ds.dims)}")
/tmp/ipykernel_3403848/4227912475.py:22: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dimensões subset: {dict(ds_sub.dims)}")


In [14]:
# CÉLULA 9: Download de teste completo
print("="*60)
print("DOWNLOAD DE TESTE")
print("="*60)

output_file = "./teste_pace_chl.nc"

try:
    print("\n1. Buscando dados...")
    results = earthaccess.search_data(
        short_name="PACE_OCI_L3M_CHL",
        temporal=("2024-06-15", "2024-06-15"),
        granule_name="*.DAY.*.0p1deg.*"
    )
    print(f"   Encontrados: {len(results)}")
    
    print("\n2. Abrindo arquivo remoto...")
    fileset = earthaccess.open(results)
    print(f"   Fileset: {len(fileset)} arquivo(s)")
    
    print("\n3. Lendo com xarray...")
    ds = xr.open_dataset(fileset[0])
    print(f"   Dims: {dict(ds.dims)}")
    
    print("\n4. Aplicando subset...")
    ds_sub = ds.sel(
        lat=slice(50, 30),
        lon=slice(-80, -60)
    )
    print(f"   Subset dims: {dict(ds_sub.dims)}")
    
    print("\n5. Carregando dados na memória...")
    ds_sub.load()
    print(f"   ✓ Carregado: {ds_sub.nbytes / 1e6:.1f} MB")
    
    print(f"\n6. Salvando em {output_file}...")
    ds_sub.to_netcdf(output_file)
    print(f"   ✓ Salvo!")
    
    ds_sub.close()
    ds.close()
    
    # Verify
    import os
    size_mb = os.path.getsize(output_file) / 1e6
    print(f"\n" + "="*60)
    print(f"✓ DOWNLOAD COMPLETO!")
    print(f"  Arquivo: {output_file}")
    print(f"  Tamanho: {size_mb:.1f} MB")
    print("="*60)
    
except Exception as e:
    print(f"\n✗ ERRO: {e}")
    import traceback
    traceback.print_exc()

DOWNLOAD DE TESTE

1. Buscando dados...
   Encontrados: 1

2. Abrindo arquivo remoto...


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

   Fileset: 1 arquivo(s)

3. Lendo com xarray...
   Dims: {'lat': 1800, 'lon': 3600, 'rgb': 3, 'eightbitcolor': 256}

4. Aplicando subset...
   Subset dims: {'lat': 200, 'lon': 200, 'rgb': 3, 'eightbitcolor': 256}

5. Carregando dados na memória...
   ✓ Carregado: 0.2 MB

6. Salvando em ./teste_pace_chl.nc...

✗ ERRO: NetCDF: HDF error


/tmp/ipykernel_3403848/3984861058.py:23: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"   Dims: {dict(ds.dims)}")
/tmp/ipykernel_3403848/3984861058.py:30: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"   Subset dims: {dict(ds_sub.dims)}")
Traceback (most recent call last):
  File "/home/desenvolvedor/projetos/hackweek/2026-proj-Trawling4PACE/.venv/lib/python3.12/site-packages/xarray/backends/writers.py", line 441, in to_netcdf
    dump_to_store(
  File "/home/desenvolvedor/projetos/hackweek/2026-proj-Trawling4PACE/.venv/lib/python3.12/site-packages/xarray/backends/writers.p

---

## 📋 Resumo do Diagnóstico

Se você chegou até aqui sem erros, o download está funcionando!

### Problemas comuns:

| Erro | Solução |
|------|--------|
| `'NoneType' object has no attribute 'open'` | Autenticação falhou. Execute `earthaccess.login(strategy='interactive')` |
| `HTTP 401 Unauthorized` | Credenciais incorretas ou expiradas |
| `0 granules found` | Data sem dados. PACE começou em março 2024 |
| `Connection timeout` | Problema de rede |

### Datas válidas:
- PACE OCI: A partir de **março 2024**
- Dados diários (DAY) disponíveis para a maioria das datas

In [22]:
# Célula de teste
import xarray as xr
import earthaccess

# Buscar e abrir
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL",
    temporal=("2024-06-15", "2024-06-15"),
    granule_name="*.DAY.*.0p1deg.*"
)
fileset = earthaccess.open(results)
ds = xr.open_dataset(fileset[0])

# Subset
ds_sub = ds.sel(lat=slice(50, 30), lon=slice(-80, -60))
ds_sub.load()

# LIMPAR ENCODING
for var in list(ds_sub.data_vars) + list(ds_sub.coords):
    if var in ds_sub:
        ds_sub[var].encoding.clear()

# Salvar
ds_sub.to_netcdf("teste_pace_chl.nc")
print("✓ Salvo!")

ds.close()

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: NetCDF: HDF error

In [23]:
# Depois do load(), antes do to_netcdf():
ds_sub.load()

# Limpar encoding problemático
for var in ds_sub.data_vars:
    ds_sub[var].encoding.clear()
for coord in ds_sub.coords:
    ds_sub[coord].encoding.clear()

# Agora salvar
ds_sub.to_netcdf(output_file)

PermissionError: [Errno 13] Permission denied: '/home/desenvolvedor/projetos/hackweek/2026-proj-Trawling4PACE/contributor_folders/leandro/teste_pace_chl.nc'

In [24]:
# Depois do load(), antes do to_netcdf():
ds_sub.load()

# Limpar encoding problemático
for var in ds_sub.data_vars:
    ds_sub[var].encoding.clear()
for coord in ds_sub.coords:
    ds_sub[coord].encoding.clear()

# Agora salvar
ds_sub.to_netcdf(output_file)

RuntimeError: NetCDF: HDF error

In [26]:
import xarray as xr
import earthaccess
import numpy as np

# Buscar e abrir
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL",
    temporal=("2024-06-15", "2024-06-15"),
    granule_name="*.DAY.*.0p1deg.*"
)
fileset = earthaccess.open(results)
ds = xr.open_dataset(fileset[0])

# Subset
ds_sub = ds.sel(lat=slice(50, 30), lon=slice(-80, -60))

# Extrair apenas chlor_a (ignorar palette que é só para visualização)
chlor_a = ds_sub['chlor_a'].values  # numpy array
lat = ds_sub['lat'].values
lon = ds_sub['lon'].values

# Fechar conexão remota
ds.close()

# Criar dataset NOVO do zero
ds_new = xr.Dataset(
    data_vars={
        'chlor_a': (['lat', 'lon'], chlor_a)
    },
    coords={
        'lat': lat,
        'lon': lon
    },
    attrs={'source': 'NASA PACE OCI L3M', 'date': '2024-06-15'}
)

# Salvar
ds_new.to_netcdf("teste_pace_chl.nc")
print("✓ Salvo!")

# Verificar
import os
print(f"Tamanho: {os.path.getsize('teste_pace_chl.nc') / 1e6:.2f} MB")

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: NetCDF: HDF error

In [27]:
import earthaccess
import xarray as xr
import os

# 1. Buscar
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL",
    temporal=("2024-06-15", "2024-06-15"),
    granule_name="*.DAY.*.0p1deg.*"
)

# 2. DOWNLOAD do arquivo completo (método correto!)
downloaded_files = earthaccess.download(results, local_path="./pace_data")
print(f"Baixado: {downloaded_files}")

# 3. Abrir arquivo LOCAL e fazer subset
ds = xr.open_dataset(downloaded_files[0])
ds_sub = ds.sel(lat=slice(50, 30), lon=slice(-80, -60))

# 4. Salvar subset (agora funciona porque abrimos arquivo local)
ds_sub.to_netcdf("./pace_data/pace_chl_subset_20240615.nc")
print("✓ Subset salvo!")

ds.close()

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Baixado: [PosixPath('pace_data/PACE_OCI.20240615.L3m.DAY.CHL.V3_1.chlor_a.0p1deg.nc')]


OSError: [Errno -101] NetCDF: HDF error: '/home/desenvolvedor/projetos/hackweek/2026-proj-Trawling4PACE/contributor_folders/leandro/pace_data/PACE_OCI.20240615.L3m.DAY.CHL.V3_1.chlor_a.0p1deg.nc'

In [28]:
import xarray as xr

# Usar h5netcdf em vez de netCDF4
ds = xr.open_dataset(
    "./pace_data/PACE_OCI.20240615.L3m.DAY.CHL.V3_1.chlor_a.0p1deg.nc",
    engine="h5netcdf"
)
print(f"✓ Aberto! Dims: {dict(ds.sizes)}")

# Subset
ds_sub = ds.sel(lat=slice(50, 30), lon=slice(-80, -60))

# Salvar (também com h5netcdf)
ds_sub.to_netcdf("./pace_data/pace_chl_subset_20240615.nc", engine="h5netcdf")
print("✓ Subset salvo!")

ds.close()

✓ Aberto! Dims: {'lat': 1800, 'lon': 3600, 'rgb': 3, 'eightbitcolor': 256}
✓ Subset salvo!
